In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Detect Beats + Tempo w/ Librosa

In [ ]:
# Load audio (mono for simplicity)
# filename = "../Songs/rufus_du_sol--innerbloom.mp3"
filename = "../Songs/omah_lay--holy_ghost.mp3"
y, sr = librosa.load(filename, sr=None)

# Tempo & beat tracking
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
tempo = np.squeeze(tempo).item()   
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

print(f"Estimated tempo: {tempo:.2f} BPM")
print("Beat times (s):", beat_times[:20], "...")

# Visualize Beat Detection

In [ ]:
# --- USER SETTINGS FOR VISUALIZATION WINDOW ---
start_time = 90.0  # Start time in seconds
window_length = 20.0  # Window length in seconds
end_time = start_time + window_length

print(f"\nVisualization window: {start_time}s to {end_time}s")

# Filter beats within the specified window
beats_in_window = beat_times[(beat_times >= start_time) & (beat_times <= end_time)]
print(f"Beats in window: {len(beats_in_window)}")



### PLOT 1 - Beat Times overlayed on raw Audio Form ###
plt.figure(figsize=(16, 5))

# Extract audio segment for the specified window
start_sample = int(start_time * sr)
end_sample = int(end_time * sr)
y_window = y[start_sample:end_sample]

# Plot windowed waveform
time_axis_window = np.linspace(start_time, end_time, len(y_window))
plt.plot(time_axis_window, y_window, alpha=1, color='blue', linewidth=0.8)

# Overlay beat markers on waveform
plt.vlines(beats_in_window, np.min(y_window), np.max(y_window), 
           color='red', alpha=0.4, linestyle='-', 
           linewidth=2, label='Beat markers')

plt.xlim(start_time, end_time)
plt.ylim([np.min(y_window) * 1.1, np.max(y_window) * 1.1])
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title(f'Audio Waveform with Beat Detection ({start_time}-{end_time}s)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()




### PLOT 2 - Time intervals between Beat Frames
beat_intervals = np.diff(beat_times)
plt.figure(figsize=(12, 4))
plt.plot(beat_times[1:], beat_intervals, 'o-', alpha=0.7, markersize=3)

# Highlight the window region
window_mask = (beat_times[1:] >= start_time) & (beat_times[1:] <= end_time)
if np.any(window_mask):
    plt.plot(beat_times[1:][window_mask], beat_intervals[window_mask], 
             'ro-', alpha=0.9, markersize=4, label=f'Window ({start_time}-{end_time}s)')

plt.axhline(y=60/tempo, color='red', linestyle='--', alpha=0.8, 
            label=f'Expected interval: {60/tempo:.3f}s')
plt.axvspan(start_time, end_time, alpha=0.2, color='yellow', label='Analysis window')
plt.xlabel('Time (s)')
plt.ylabel('Beat Interval (s)')
plt.title('Beat Interval Analysis (Full Song)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()